In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import os
from matplotlib import cm
from matplotlib import gridspec
from mayavi import mlab
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)

#Folders
projpath    = os.path.realpath("..")
pathvectors = os.path.join(projpath, "Results", "Vectors")
pathfacepca  = os.path.join(projpath, "DataBases", "FacePCA")

#Read data
#os.chdir(pathdbases)
#ibd_fs = pd.read_csv("IBD_FS.csv", sep=",")

#os.chdir(pathmerged)
#mergeddat = pd.read_csv("MergedDat.csv", sep=",")

os.chdir(pathvectors)
sex_landmarks_nonallo = np.matrix(pd.read_csv("sex_landmarks_nonallo.csv", sep=","))
sex_landmarks_total   = np.matrix(pd.read_csv("sex_landmarks_total.csv", sep=","))
sex_landmarks_groups  = pd.read_csv("sex_landmarks_groups.csv", sep=",")

os.chdir(pathfacepca)
#eigenvectors = pd.read_csv("eigenvectors.csv", sep=",", header=None)
#means        = pd.read_csv("means.txt", sep=",", header=None)
facets       = pd.read_csv("facets.csv", sep=",", header=None)

#Getting landmarks from PCA scores
#sex_landmarks = ( ( np.matrix(sex_vectors_total.iloc[:,0:70]) * 1)  * np.matrix(eigenvectors).transpose() ) + np.matrix(means).transpose()

#Getting landmarks for extreme PCs
#PCs = np.zeros([4,35])
#PCs[0][0] = (mergeddat.std()*3)[0]
#PCs[1][0] = -(mergeddat.std()*3)[0]
#PCs[2][1] = (mergeddat.std()*3)[1]
#Cs[3][1] = -(mergeddat.std()*3)[1]
#PCs_landmarks = ( np.matrix(PCs) * np.matrix(eigenvectors).transpose() ) + np.matrix(means).transpose()
#avg_landmarks = (np.matrix(np.zeros(35)) * np.matrix(eigenvectors).transpose() ) + np.matrix(means).transpose()


In [2]:
#### FUNCTIONS ######
def get_euc_dist(landmark1, landmark2):
    '''
    Get the Euclidean distances between to set of 3D landmarks
    Usage
        Input:
            - landmark1 and landmark2: set of 3D landmarks in one single row, in X,Y,Z consecutive fashion, to compute the distances from
        Output:
            - euc_dist: euclidean distance between corresponding set of 3D landmarks
    '''
    #Importing libraries
    import numpy as np
    import pandas as pd
    
    #Setting number of landmarks
    n_landmarks = int(landmark1.shape[1] / 3)
    
    #Reshaping landmarks
    face1 = landmark1.reshape(n_landmarks, 3)
    face2 = landmark2.reshape(n_landmarks, 3)
    #Calculating euclidean distance
    euc_dist = np.array(np.sqrt(np.sum(np.power(face1 - face2, 2), axis=1))).flatten()
    return(euc_dist)

def get_norm_disp(landmark1, landmark2, facets):
    '''
    Get the normal displacement between set of 3D landmarks
    Usage
        Input:
            - landmark1 and landmark2: set of 3D landmarks in one single row, in X,Y,Z consecutive fashion, to compute the distances from
            - facets: facets connecting the 3D point to form a polygon
        Output:
            - norm_disp: normal displacement
    '''
    
    #Importing libraries
    import numpy as np
    import pandas as pd
    
    facets = np.matrix(facets)
    
    #Setting number of landmarks
    n_landmarks = int(landmark1.shape[1] / 3)
    
    #Reshaping landmarks
    face1 = landmark1.reshape(n_landmarks, 3)
    face2 = landmark2.reshape(n_landmarks, 3)
    
    #Create a zeroed array with the same type and shape as our vertices i.e., per vertex normal
    norm = np.zeros( face1.shape, dtype=face1.dtype )
    
        

In [9]:
euc_dist= get_euc_dist(sex_landmarks_nonallo[0], sex_landmarks_nonallo[4])

In [82]:
#Comparing METHODS

#METHOD 1 (https://sites.google.com/site/dlampetest/python/calculating-normals-of-a-triangle-mesh-using-numpy)
def normalize_v3(arr):
    ''' 
    Normalize a numpy array of 3 component vectors shape=(n,3) 
    '''
    lens = np.sqrt( arr[:,0]**2 + arr[:,1]**2 + arr[:,2]**2 )
    arr[:,0] /= lens
    arr[:,1] /= lens
    arr[:,2] /= lens                
    return(arr)

norm = np.zeros( face1.shape, dtype=face1.dtype )
n_landmarks = int(sex_landmarks_nonallo[0].shape[1] / 3)
face1 = sex_landmarks_nonallo[0].reshape(n_landmarks, 3)
face2 = sex_landmarks_nonallo[4].reshape(n_landmarks, 3)
faces = np.matrix(facets)
tris = face1[faces-1]
#Calculate the normal for all the triangles, by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle             
n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
normalize_v3(n)
norm[ faces[:,0].flatten() -1 ] += n
norm[ faces[:,1].flatten() -1 ] += n
norm[ faces[:,2].flatten() -1 ] += n
normalize_v3(norm)

array([[-0.97587041, -0.20695325,  0.06962258],
       [-0.97371963, -0.22122659,  0.05411903],
       [-0.97066445, -0.23717746,  0.03946359],
       ...,
       [ 0.97392388, -0.16258908,  0.15823105],
       [ 0.97329931, -0.14644303,  0.17675658],
       [ 0.97221138, -0.13300404,  0.19265245]])

In [114]:
vertices = np.array([[ 0.82667452,  0.89591247,  0.91638623],
                        [ 0.10045271,  0.50575086,  0.73920507],
                        [ 0.06341482,  0.17413744,  0.6316301 ],
                        [ 0.75613029,  0.82585983,  0.10012549],
                        [ 0.45498342,  0.5636221 ,  0.10940527],
                        [ 0.46079863,  0.54088544,  0.1519899 ],
                        [ 0.61961934,  0.78550213,  0.43406491],
                        [ 0.12654252,  0.7514213 ,  0.18265301],
                        [ 0.94441365,  0.00428673,  0.46893573],
                        [ 0.79083297,  0.70198129,  0.75670947]] )
faces = np.array( [ [0,1,2],
                       [0,2,3], 
                       [1,2,3], 
                       [1,4,5], 
                       [2,5,6], 
                       [6,3,7], 
                       [9,8,7] ] )

#Create a zeroed array with the same type and shape as our vertices i.e., per vertex normal
norm = np.zeros( vertices.shape, dtype=vertices.dtype )
#Create an indexed view into the vertex array using the array of three indices for triangles
tris = vertices[faces]
#Calculate the normal for all the triangles, by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle             
n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
# n is now an array of normals per triangle. The length of each normal is dependent the vertices, 
# we need to normalize these, so that our next step weights each normal equally.
normalize_v3(n)
# now we have a normalized array of normals, one per triangle, i.e., per triangle normals.
# But instead of one per triangle (i.e., flat shading), we add to each vertex in that triangle, 
# the triangles' normal. Multiple triangles would then contribute to every vertex, so we need to normalize again afterwards.
# The cool part, we can actually add the normals through an indexed view of our (zeroed) per vertex normal array
norm[ faces[:,0] ] += n
norm[ faces[:,1] ] += n
norm[ faces[:,2] ] += n
normalize_v3(norm)

array([[ 0.68647611, -0.72714585,  0.00307691],
       [-0.41663866, -0.79746992,  0.4364103 ],
       [ 0.561194  , -0.48419887,  0.67127696],
       [ 0.53644249,  0.60476166,  0.58863638],
       [-0.49963522, -0.79064121, -0.35390836],
       [ 0.17280246, -0.97347702, -0.14993929],
       [ 0.84690504,  0.45590931,  0.27367601],
       [ 0.61293173,  0.4128088 , -0.67372367],
       [ 0.61293173,  0.4128088 , -0.67372367],
       [ 0.61293173,  0.4128088 , -0.67372367]])

In [112]:
#METHOD 2 (https://www.mathworks.com/matlabcentral/fileexchange/24330-patch-normals)

# Get all edge vectors
tris = face1[faces-1]
e1 = tris[:,:,0] - tris[:,:,1]
e2 = tris[:,:,1] - tris[:,:,2]
e3 = tris[:,:,0] - tris[:,:,2]
#e1 = face1[ faces[:,0].flatten() -1 ,:] - face1[ faces[:,1].flatten() -1 ,:];
#e2 = face1[ faces[:,1].flatten() -1 ,:] - face1[ faces[:,2].flatten() -1 ,:];
#e3 = face1[ faces[:,2].flatten() -1 ,:] - face1[ faces[:,0].flatten() -1 ,:];

# Normalize edge vectors
e1_norm = e1 ./ repmat(sqrt(e1(:,1).^2 + e1(:,2).^2+e1(:,3).^2),1,3); 
e2_norm = e2 ./ repmat(sqrt(e2(:,1).^2 + e2(:,2).^2+e2(:,3).^2),1,3); 
e3_norm = e3 ./ repmat(sqrt(e3(:,1).^2 + e3(:,2).^2+e3(:,3).^2),1,3);

In [111]:
e1

matrix([[-0.0070817 , -0.00739693, -0.00718243],
        [-0.0070817 , -0.00768736, -0.00739693],
        [-0.00669199, -0.00696082, -0.00646323],
        ...,
        [-0.01503748, -0.01533616, -0.01467541],
        [-0.01533616, -0.01499108, -0.01467541],
        [-0.01499108, -0.01462813, -0.01467541]])

In [100]:
faces[:,0]

matrix([[3688],
        [3688],
        [3684],
        ...,
        [  87],
        [  71],
        [  80]])